In [1]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from textblob import TextBlob
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

c:\Users\markham\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [ ]:
test_data = pd.read_csv('data/processed_articles_2021_V1.csv')
test_data['content'] = test_data['title'] + test_data['body']
test_data = test_data.drop(columns=['title','body'])
test_data['date'] = pd.to_datetime(test_data['date'])
test_data = test_data.sort_values(by='date')
test_data['content'] = test_data['content'].apply(lambda x: ' '.join(x.split(',')))
stop_words = set(stopwords.words('english'))
stop_words_list = [ 'msci','iran','states','italy','united','china','usa','us','america','american','americans','chinese','china','russia','russian','putin','vladimir','trump','donald','biden','joe','ukraine','ukrainian','ukrainians','ukraines','ukraine','say','jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec','mon','tue','wed','thu','fri','sat','sun','monday','tuesday','wednesday','thursday','friday','saturday','sunday','january','february','march','april','may','june','july','august','september','october','november','december','today','yesterday','tomorrow','week','month','year','time','day','weekend','morning','afternoon','evening','night','news','new','news']
list_numbers = ['liga','len','nhl','one','two','three','four','five','six','seven','eight','nine','ten','hundred','thousand','million','billion','trillion']
country_list = ['China', 'United States of America', 'United Kingdom', 'France', 'Germany', 'Japan', 'Russia', 'Australia', 'Canada', 'India', 'Brazil', 'Italy', 'Spain', 'South Korea', 'Mexico', 'Netherlands', 'Switzerland', 'Sweden', 'Norway', 'Denmark', 'Finland', 'Greece', 'Ireland', 'Portugal', 'Poland', 'Ukraine', 'Romania', 'Belgium', 'Austria', 'Turkey', 'Saudi Arabia', 'United Arab Emirates', 'Iran', 'Iraq', 'Israel', 'Egypt', 'South Africa', 'Argentina', 'Venezuela', 'Thailand', 'Malaysia', 'Singapore', 'Indonesia', 'Philippines', 'Pakistan', 'Bangladesh', 'Nigeria', 'Kenya', 'Tanzania', 'Uganda']
stop_words.update(stop_words_list)
stop_words.update(list_numbers)
stop_words.update(country_list)
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if word.isalpha()]))
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if word != 'nan']))
test_data['content_tokens'] = test_data['content'].apply(lambda x: x.split())

In [3]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score

# Define parameter search space
vector_sizes = [30, 40, 50, 60, 70, 80, 90, 100]
windows = [3, 4, 5, 6, 7, 8, 9, 10]
min_counts = [2, 3, 4, 5, 6, 7]
Ns = [10, 20, 30, 40, 50]

best_silhouette_params = None
best_silhouette_score = -1

best_ch_params = None
best_ch_score = -1

for vector_size in vector_sizes:
    for window in windows:
        for min_count in min_counts:
            for N in Ns:
                # Step 2: Train Word2Vec model on tokenized words
                model = Word2Vec(sentences=test_data['content_tokens'], vector_size=vector_size, window=window, min_count=min_count, workers=8)
                
                # Step 3: Compute TF-IDF weights for all words in the dataset
                corpus = test_data['content'].tolist()
                vectorizer = TfidfVectorizer()
                tfidf_matrix = vectorizer.fit_transform(corpus)
                word2tfidf = dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))
                
                # Step 4: Identify high-frequency words per article
                test_data['important_words'] = test_data['content_tokens'].apply(
                    lambda tokens: sorted(
                        set(tokens),  # Remove duplicates
                        key=lambda x: word2tfidf.get(x, 0),  # Rank by TF-IDF score
                        reverse=False  # Lower TF-IDF means higher frequency in this case
                    )[:N]
                )
                
                # Step 5: Convert text to weighted vectors
                def text_to_vector(text, model, word2tfidf):
                    vectors = []
                    weights = []
                    for word in text:
                        if word in model.wv:
                            vectors.append(model.wv[word])
                            weights.append(word2tfidf.get(word, 1.0))
                    if not vectors:
                        return np.zeros(model.vector_size)
                    vectors = np.array(vectors)
                    weights = np.array(weights) / sum(weights)
                    return np.average(vectors, axis=0, weights=weights)
                
                test_data['vector'] = test_data['important_words'].apply(lambda x: text_to_vector(x, model, word2tfidf))
                
                # Step 6: Prepare data for clustering
                X = np.array(test_data['vector'].tolist())
                
                # Step 7: Perform clustering
                num_clusters = 2
                kmeans = KMeans(n_clusters=num_clusters, random_state=42)
                labels = kmeans.fit_predict(X)
                test_data['cluster_label'] = labels
                
                # Step 8: Evaluate clustering
                if len(set(labels)) > 1:
                    silhouette = silhouette_score(X, labels)
                    ch_score = calinski_harabasz_score(X, labels)
                    
                    if silhouette > best_silhouette_score:
                        best_silhouette_score = silhouette
                        best_silhouette_params = (vector_size, window, min_count, N)
                    
                    if ch_score > best_ch_score:
                        best_ch_score = ch_score
                        best_ch_params = (vector_size, window, min_count, N)
                        
print(f"Best Silhouette parameters: Vector Size={best_silhouette_params[0]}, Window={best_silhouette_params[1]}, Min Count={best_silhouette_params[2]}, N={best_silhouette_params[3]}")
print(f"Best Calinski-Harabasz parameters: Vector Size={best_ch_params[0]}, Window={best_ch_params[1]}, Min Count={best_ch_params[2]}, N={best_ch_params[3]}")


Best Silhouette parameters: Vector Size=100, Window=8, Min Count=2, N=30
Best Calinski-Harabasz parameters: Vector Size=100, Window=8, Min Count=2, N=30
